# Doc2VecでWikipedia全文を使用

Wiki全文の一部を、Doc2Vecのインプットに指定し、動作確認します。

今回は２つの手法を比較しました。

- PV-DBOW

- PV-DM

いずれも、質問文に対する予測結果が、学習セットと近似しているとは言い難い結果となりました。

## (1) Wikipediaコンテンツファイルから本文を抽出

<a href="31-Wikipedia-contents-csv.ipynb"><b>こちらの手順</b></a> にて、いったんローカルPCにCSVファイル化しておきます。



## (2) Wikipedia全文のTaggedDocumentを生成

Wikipedia全文のCSVファイルからデータフレームを読み込み、TaggedDocumentに変換します。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
'''
    Wikipedia全文から生成されたCSVを読み込み、
    配列を生成する
'''
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'
csv_file_name = os.path.join(extracted_dir_path, 'Wikipedia-content.csv')
df = pd.read_csv(csv_file_name)

wiki_ids = np.array(df['id'])
wiki_titles = np.array(df['title'])
wiki_contents = np.array(df['content'])

In [3]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [4]:
from gensim.models.doc2vec import TaggedDocument
# from learning.core.nlang import Nlang

def get_tagged_document_list(wiki_ids, wiki_contents):
    '''
        タグとコーパスをTaggedDocumentに設定
    '''
    tagged_document_list = []

    for index, _ in enumerate(wiki_ids):
        tag = wiki_ids[index]
        sentences = wiki_contents[index]

        # 品詞は落とさない様にする
        separated_sentences = Nlang_naive.split(sentences)
        words = separated_sentences.split()
        tagged_document_list.append(TaggedDocument(words=words, tags=[tag]))
        
    return tagged_document_list

In [5]:
'''
    時間がかかるので一部のみ使用します（全量＝100万件）
'''
partial_cnt = 1000

wiki_ids = wiki_ids[0:partial_cnt]
wiki_contents = wiki_contents[0:partial_cnt]

In [6]:
wiki_tagged_document_list = get_tagged_document_list(wiki_ids, wiki_contents)

生成されたTaggedDocumentの一例をみてみます。

品詞を落としていないせいか、かなり大きなコーパスであることが確認できます。

（実は下記の例でも小さい方）

In [7]:
wiki_tagged_document_list[0]

TaggedDocument(words=['アンパサンド', '（，', '＆）', 'と', 'は', 'と', 'を', '意味', 'する', '記号', 'だ', 'ある', '英語', 'の', 'に', '相当', 'する', 'ラテン語', 'の', 'の', '合', '字', 'で', '（', 'ｅｔ', 'ｃｅｔｅｒａ', '＝', 'ａｎｄ', 'ｓｏ', 'ｆｏｒｔｈ', '）', 'を', 'と', '記述', 'する', 'こと', 'が', 'ある', 'の', 'は', 'その', 'ため', 'Ｔｒｅｂｕｃｈｅｔ', 'ＭＳ', 'フォント', 'で', 'は', 'と', '表示', 'する', 'れる', '”', 'ｅｔ', '”', 'の', '合', '字', 'だ', 'ある', 'こと', 'が', '容易', 'に', 'わかる', 'その', '使用', 'は', '１', '世紀', 'に', '遡る', 'こと', 'が', 'できる', '（', '１', '）、', '５', '世紀', '中葉', '（', '２', '，', '３', '）', 'から', '現代', '（', '４', '−', '６', '）', 'に', '至る', 'まで', 'の', '変遷', 'が', 'わかる', 'Ｚ', 'に', '続く', 'ラテン', '文字', 'アルファベット', 'の', '２７', '字', '目', 'と', 'する', 'れる', 'た', '時期', 'も', 'ある', 'アンパサンド', 'と', '同じ', '役割', 'を', '果たす', '文字', 'に', 'の', 'ｅｔ', 'と', '呼ぶ', 'れる', '数字', 'の', '７', 'に', '似る', 'た', '記号', 'が', 'ある', 'た', '（，', 'Ｕ', '＋', '２０４', 'Ａ', '）。', 'この', '記号', 'は', '現在', 'も', 'ゲール', '文字', 'で', '使う', 'れる', 'て', 'いる', '記号', '名', 'の', 'アンパサンド', 'は', 'ラテン語', 'まじる', 'の', '英語', '「＆', 'は', 'それ', 

## (3) 学習実行

### (3-1) Wikipedia全文のTaggedDocumentだけをインプットとして学習

まずはマイオペのテストデータは入れないでテストします。

概ね下記のようなコードを実行して学習します。

分類数は不明なので、適当なサイズに設定しておきます（たとえばサンプルの10分の1とか）。

繰り返し回数はサイズの10倍を指定します。

In [8]:
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

def build_vocabulary(tagged_document_list, dm=0, size=100):
    '''
        tagged_document_listは、
        すべてのWikipediaのタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
    '''
    model = Doc2Vec(dm=dm, size=size, min_count=1, iter=size*10)
    model.build_vocab(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('build_vocabulary: model saved temporary')

    return model

def train(model, tagged_document_list, dm=0):
    '''
        tagged_documentが収容されたリストを引数にして、
        学習を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        （今回は比較のため、両方で試す）
    '''
    ret = model.train(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('train: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [9]:
'''
    tagged_document_listを使用し、学習実行（PV-DBOW）
'''
model_dbow = build_vocabulary(wiki_tagged_document_list, dm=0)

build_vocabulary: model saved temporary


In [10]:
train(model_dbow, wiki_tagged_document_list, dm=0) 

train: document vector size=1000, return=1193280113


1193280113

In [11]:
'''
    tagged_document_listを使用し、学習実行（PV-DM）
'''
model_dm = build_vocabulary(wiki_tagged_document_list, dm=1)

build_vocabulary: model saved temporary


In [12]:
train(model_dm, wiki_tagged_document_list, dm=1) 

train: document vector size=1000, return=1193261421


1193261421

## (3) 予測実行

作成されたモデルを使用して予測を実行する関数になります。

In [13]:
from gensim import models

def predict(word, dm=0):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(dm))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

def get_title_from_tag(tag):
    for index, _ in enumerate(wiki_ids):
        if wiki_ids[index] == tag:
            return wiki_titles[index]
        
    return None

def print_predict_result(corpus, similarities):
    print(corpus)

    for tag, similarity in similarities:
        print(get_title_from_tag(tag), similarity)

### (3-1) 適当な質問文で予測（PV-DBOW)

In [14]:
'''
    単語数がやや多い質問文を使い、質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=0)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
ヘラクレイトス 0.4563015401363373
ワンダフルライフ (映画) 0.40719297528266907
安達哲 0.40467700362205505
社会学者の一覧 0.4030371308326721
築山 0.40055203437805176
風邪 0.39788639545440674
ゲーム 0.39261096715927124
消滅した政権一覧 0.39236465096473694
医療漫画 0.3832259178161621
加瀬邦彦 0.3791019320487976


元ねたと答え合わせ。

元データには、２件ほど本文にキーワードがあるもの（id=329[風邪]と、id=655[水木しげる]）があります。

ただし、予測結果（１件ヒット）としては類似度が低い感じです。
```
<doc id="329" url="https://ja.wikipedia.org/wiki?curid=329" title="風邪">
風邪（かぜ、common cold, nasopharyngitis, rhinopharyngitis, acute coryza, a cold）とは、ウイルスによる上気道感染症であり（以下略）

<doc id="655" url="https://ja.wikipedia.org/wiki?curid=655" title="水木しげる">
（中略）帰還してまもなく行軍中に風邪を引いた際にマラリアを発症（以下略）

```

In [17]:
'''
    タイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=0)
print_predict_result(corpus, ret)

['風邪']
魔少年ビーティー 0.4811190962791443
ウード 0.4510417580604553
日本の漫画作品一覧 0.4497838616371155
かくれんぼ 0.44944673776626587
仮面ライダーV3 0.4454251527786255
はしもとみつお 0.4416940212249756
現代洋子 0.4305399954319
出口竜正 0.42982280254364014
大橋薫 0.42832446098327637
くじらいいく子 0.42305099964141846


「風邪」が記述されているドキュメントに１件もヒットしませんでした。

### (3-2) 適当な質問文で予測（PV-DM)

In [15]:
'''
    単語数がやや多い質問文を使い、質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=1)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
坂本タクマ 0.35875970125198364
共通言語ランタイム 0.35062533617019653
12月8日 0.35055220127105713
風邪 0.33461111783981323
みやすのんき 0.32742440700531006
吉田里深 0.32369211316108704
社会学者の一覧 0.3209966719150543
イッセー尾形 0.31948035955429077
元号から西暦への変換表 0.31912702322006226
ゲームのタイトル一覧 0.3180711269378662


「風邪」が記述されているドキュメントに１件ヒットしました（ただし類似度は低い）

In [16]:
'''
    タイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=1)
print_predict_result(corpus, ret)

['風邪']
風邪 0.5769541263580322
こしたてつひろ 0.5401366949081421
野崎ふみこ 0.5135844945907593
石井克人 0.5127018690109253
幡地英明 0.5066384673118591
是枝裕和 0.5041813254356384
田島みるく 0.4961746633052826
2003年 0.49040836095809937
高瀬由香 0.48796403408050537
のなかみのる 0.48767614364624023


「風邪」が記述されているドキュメントにかかってきました。

PV-DMはこちらのような使い勝手にフィットしているのかもしれません。